In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate('serviceAccountKey.json')
firebase_admin.initialize_app(cred)

In [18]:
import cv2
import datetime

cap = cv2.VideoCapture(0)

# Initialize background subtractor
fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()
    
    # Apply background subtractor to get foreground mask
    fgmask = fgbg.apply(frame)
    
    # Threshold mask to get binary motion mask
    thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)[1]
    
    # Display the frame and threshold images    
    cv2.imshow('Original', frame)
    cv2.imshow('Thresholded', thresh)

    
    k = cv2.waitKey(30) & 0xff
    if k == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [19]:
import cv2
import datetime
import base64
import numpy as np
from pytz import timezone 

db = firestore.client()

cam = cv2.VideoCapture(0)

fgbg = cv2.createBackgroundSubtractorMOG2()
c=0
_, first_frame = cam.read()
first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY) 
first_gray = cv2.GaussianBlur(first_gray, (5, 5), 0)

while True:
    _, frame = cam.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    diff = cv2.absdiff(first_gray, gray)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    cv2.imshow('Threshold Frame', thresh)
    cv2.imshow('Normal Frame', frame)
    
    if cv2.countNonZero(thresh) > 8000:
        current_time = datetime.datetime.now(timezone("Asia/Kolkata"))
    
        filename = current_time.strftime("%Y%m%d-%H%M%S") + '.jpg'
        cv2.imwrite(filename, frame)

        doc_ref = db.collection('notifications').document()

        _, buffer = cv2.imencode('.jpg',frame)
        base64_str = base64.b64encode(buffer).decode('utf-8')
        doc_ref.set({
                'time':current_time,
                'Motion ID': 1255,
                'cam_proof':"data:image/jpeg;base64,"+base64_str,
                'type': "motion"
        })
    
        cam.release()
        cv2.destroyAllWindows()
        break